In [34]:
import json
import os
from datetime import datetime
import requests
import sys
from pathlib import Path
from extract_markdown import extract_markdown_from_pdf, extract_from_to, count_pages
from create_embeddings import extract_pdfs_to_markdown,generate_faiss_embeddings

Sample Testing

In [35]:

pdf_path = r"dof\2025\01\02012025-MAT.pdf"  

num_pages = count_pages(pdf_path)
print(f"Número de páginas en el PDF: {num_pages}")



Número de páginas en el PDF: 166


In [39]:
markdown_text = extract_markdown_from_pdf(pdf_path)
print(markdown_text[:1000])  


ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

In [ ]:
md_path = "extract/documento.md"  
extract_from_to(pdf_path, md_path)
print(f"Markdown guardado en: {md_path}")


ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

Embeddings

In [ ]:

source_directory = "dof/2025"
target_directory = "extracted/2025"
extract_pdfs_to_markdown(source_directory, target_directory)

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

In [ ]:

md_directory = "extracted/2025"   # Ruta donde se guardaron los archivos .md
faiss_index_file = "faiss_index.idx"  # Archivo donde se guardará el índice FAISS

index, metadata = generate_faiss_embeddings(md_directory, faiss_index_file)

model.safetensors:  91%|#########1| 4.53G/4.96G [00:00<?, ?B/s]

Procesado: extracted/2025\01\02012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\03012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\06012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\07012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\08012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\09012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\10012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\13012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\14012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\15012025-MAT.md → Embedding generado


c:\Users\erojas\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\erojas\.cache\huggingface\hub\models--hkunlp--instructor-xl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Procesado: extracted/2025\01\16012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\17012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\20012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\21012025-MAT.md → Embedding generado
Procesado: extracted/2025\01\22012025-MAT.md → Embedding generado
Índice FAISS guardado en: faiss_index.idx
Metadatos guardados en: faiss_index.idx


In [ ]:
def search_faiss(query, top_k=5, filter_date=None, filter_category=None, filter_page=None):
    """
    Realiza una búsqueda en FAISS con la opción de filtrar por fecha, categoría o número de página.

    Args:
        query (str): La consulta en lenguaje natural.
        top_k (int): Número de resultados a devolver.
        filter_date (str, optional): Filtrar por fecha en formato "YYYY-MM-DD".
        filter_category (str, optional): Filtrar por tipo de documento (ej. "MAT").
        filter_page (str, optional): Filtrar por páginas específicas.

    Returns:
        list: Resultados con los textos y metadatos.
    """

    # ✅ Verificar si FAISS y metadatos están cargados
    if index is None or metadata is None:
        raise ValueError("⚠️ El índice FAISS o los metadatos no están cargados correctamente.")

    # ✅ Generar embedding de la consulta
    query_embedding = model.encode(query).astype("float32").reshape(1, -1)

    # ✅ Verificar que la dimensión del embedding coincide con FAISS
    if query_embedding.shape[1] != index.d:
        raise ValueError(f"⚠️ Dimensión del embedding ({query_embedding.shape[1]}) no coincide con FAISS ({index.d}).")

    # ✅ Buscar en FAISS
    distances, indices = index.search(query_embedding, top_k)

    # ✅ Recuperar resultados y aplicar filtros por metadatos
    results = []
    for i, idx in enumerate(indices[0]):
        if idx == -1 or idx >= len(metadata):  # Evitar índices fuera de rango
            continue

        doc_metadata = metadata[idx]

        # ✅ Aplicar filtros de manera segura
        if filter_date and doc_metadata.get("date") != filter_date:
            continue
        if filter_category and doc_metadata.get("category") != filter_category:
            continue
        if filter_page and filter_page not in doc_metadata.get("pages", ""):
            continue

        # ✅ Agregar al resultado
        results.append({
            "rank": len(results) + 1,  # Ajustar el ranking después del filtrado
            "file": doc_metadata.get("file", "Desconocido"),
            "date": doc_metadata.get("date", "Desconocida"),
            "category": doc_metadata.get("category", "Desconocida"),
            "pages": doc_metadata.get("pages", "Desconocidas"),
            "path": doc_metadata.get("path", "Desconocido"),
            "distance": float(distances[0][i])  # Convertir a float para evitar errores con JSON
        })

    # ✅ Ordenar resultados por relevancia (menor distancia es mejor)
    results = sorted(results, key=lambda x: x["distance"])

    return results


In [ ]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# 📌 Rutas de los archivos generados
faiss_index_file = "faiss_index.idx"
metadata_file = "faiss_index.json"
model_name = "hkunlp/instructor-xl"

# 📌 Verificar que los archivos existen
if not os.path.exists(faiss_index_file) or not os.path.exists(metadata_file):
    raise FileNotFoundError("❌ No se encontraron los archivos FAISS. Asegúrate de generar los embeddings primero.")

# 📌 Cargar el índice FAISS
index = faiss.read_index(faiss_index_file)

# 📌 Cargar los metadatos
with open(metadata_file, "r", encoding="utf-8") as f:
    metadata = json.load(f)

# 📌 Cargar el modelo de embeddings
model = SentenceTransformer(model_name)

print("✅ Índice FAISS y metadatos cargados correctamente.")



✅ Índice FAISS y metadatos cargados correctamente.


In [ ]:
query = "Reforma fiscal en México"
results = search_faiss(query, top_k=5)

for res in results:
    print(res)


{'rank': 1, 'file': '07012025-MAT.md', 'date': 'Desconocida', 'category': 'Desconocida', 'pages': 'Desconocidas', 'path': 'extracted/2025\\01\\07012025-MAT.md', 'distance': 0.5185215473175049}
{'rank': 2, 'file': '16012025-MAT.md', 'date': 'Desconocida', 'category': 'Desconocida', 'pages': 'Desconocidas', 'path': 'extracted/2025\\01\\16012025-MAT.md', 'distance': 0.529853105545044}
{'rank': 3, 'file': '10012025-MAT.md', 'date': 'Desconocida', 'category': 'Desconocida', 'pages': 'Desconocidas', 'path': 'extracted/2025\\01\\10012025-MAT.md', 'distance': 0.56873619556427}
{'rank': 4, 'file': '08012025-MAT.md', 'date': 'Desconocida', 'category': 'Desconocida', 'pages': 'Desconocidas', 'path': 'extracted/2025\\01\\08012025-MAT.md', 'distance': 0.5725093483924866}
{'rank': 5, 'file': '21012025-MAT.md', 'date': 'Desconocida', 'category': 'Desconocida', 'pages': 'Desconocidas', 'path': 'extracted/2025\\01\\21012025-MAT.md', 'distance': 0.5894616842269897}
